In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Load Data and split into Train and Test
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
Data = pd.DataFrame(data = mnist['data'], columns = mnist['feature_names'])
Target = pd.DataFrame(data = mnist['target'], columns = ['class'])
Train_Data = Data[:60000]
Test_Data = Data[60001:]
Target_train = Target[:60000]
Target_test = Target[60001:]

In [ ]:
# changing data inyo tensor objects:
Train_Data = np.array(Train_Data)
Test_Data = np.array(Test_Data)
Train_Data = Train_Data.astype(dtype = 'float64')
Test_Data = Test_Data.astype(dtype = 'float64')
Train_Data = torch.from_numpy(Train_Data)
Test_Data = torch.from_numpy(Test_Data)

Target_train = np.array(Target_train)
Target_test = np.array(Target_test)
Target_train = Target_train.astype(int)
Target_test = Target_test.astype(int)
Target_train = torch.from_numpy(Target_train)
Target_test = torch.from_numpy(Target_test)

# One hot encoding of atrget varivales
Target_test = torch.nn.functional.one_hot(Target_test)
Target_train = torch.nn.functional.one_hot(Target_train)